In [1]:
!pip install -qq ultralytics
!pip install -qq 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -qq jupyter_bbox_widget roboflow dataclasses-json supervision==0.23.0
!pip uninstall -qq setuptools -y && pip install -qq setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 20.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.9/774.9 kB 40.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.8 MB/s eta 0:00:0000:01
ERROR: pip's

In [2]:
!pip uninstall -y -qq realesrgan
!git clone -qq https://github.com/xinntao/Real-ESRGAN.git
!pip install -qqq basicsr
!pip install -r /kaggle/working/Real-ESRGAN/requirements.txt

%cd Real-ESRGAN
!python /kaggle/working/Real-ESRGAN/setup.py develop
%cd /kaggle/working/
!pip install -qqq torchvision==0.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.3 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110547 sha256=af31cc81479a80feedc4a998beeba8f9a9232e9ec6683bf095e1d9676f16fbca
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfull

In [4]:
import os
import torch
import cv2
import numpy as np
from PIL import Image
from einops import rearrange
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from ultralytics import YOLO
object_detection_model=YOLO("/kaggle/input/cs299-pre-trained-models/yolo11n_best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
from segment_anything import sam_model_registry,SamPredictor
CHECKPOINT_PATH = os.path.join("/kaggle/input/cs299-pre-trained-models/sam_vit_h_4b8939.pth")
sam=sam_model_registry['vit_h'](checkpoint=CHECKPOINT_PATH).to(device=device)
segmentation_model=SamPredictor(sam)

In [7]:
import torch.nn as nn
from torchvision import transforms

class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(256, 512, 4, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 4, stride=2, padding=1), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 512, 4, stride=2, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1), nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))
inpainting_model=Autoencoder()
inpainting_model.load_state_dict(torch.load(
    "/kaggle/input/cs299-pre-trained-models/autoencoder_epoch_500.pth",
    map_location=torch.device('cpu')
))

inpainting_transformer=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
    
])

In [8]:
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet

import sys
sys.path.append('/kaggle/working/Real-ESRGAN')

from realesrgan import RealESRGANer
from PIL import Image
import numpy as np

superresolution_model=RRDBNet(num_in_ch=3,num_out_ch=3,num_feat=64,num_block=23,num_grow_ch=32,scale=10)

superresolution_upscaler=RealESRGANer(
    scale=1000,
    model_path='/kaggle/input/cs299-pre-trained-models/RealESRGAN_x4plus.pth',
    model=superresolution_model,
    tile=0,tile_pad=100,
    pre_pad=0,half=False
)

In [9]:
from diffusers import DiffusionPipeline,EulerAncestralDiscreteScheduler
from huggingface_hub import snapshot_download

snapshot_download("sudo-ai/zero123plus-v1.1",force_download=False)

zero123plus_model= DiffusionPipeline.from_pretrained(
    "sudo-ai/zero123plus-v1.1",
    custom_pipeline="sudo-ai/zero123plus-pipeline",
    torch_dtype=torch.float16
)

zero123plus_model.scheduler=EulerAncestralDiscreteScheduler.from_config(
    zero123plus_model.scheduler.config,timestep_spacing='trailing'
)
zero123plus_model.to('cuda:0')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

inference.py:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pipeline.py:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Zero123PlusPipeline {
  "_class_name": "Zero123PlusPipeline",
  "_diffusers_version": "0.31.0",
  "_name_or_path": "sudo-ai/zero123plus-v1.1",
  "feature_extractor_clip": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "feature_extractor_vae": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "ramping_coefficients": [
    0.0,
    0.2060057818889618,
    0.18684479594230652,
    0.24342191219329834,
    0.18507817387580872,
    0.1703828126192093,
    0.15628913044929504,
    0.14174538850784302,
    0.13617539405822754,
    0.13569170236587524,
    0.1269884556531906,
    0.1200924888253212,
    0.12816639244556427,
    0.13058121502399445,
    0.14201879501342773,
    0.15004529058933258,
    0.1620427817106247,
    0.17207716405391693,
    0.18534132838249207,
    0.20002241432666779,
    0.21657466888427734,
    0.22996725142002106,
    0.24613411724567413,
    0.25141021609306335,
    0.26613450050354004,
    0.271847128868103,
    0.2850190997123718,
    0.2857492268085

In [10]:
output_path="output"

os.makedirs(os.path.join(output_path,"detected_object"),exist_ok=True)
os.makedirs(os.path.join(output_path,"cropped_object"),exist_ok=True)
os.makedirs(os.path.join(output_path,"segmented_object"),exist_ok=True)
os.makedirs(os.path.join(output_path,"inpainted_object"),exist_ok=True)
os.makedirs(os.path.join(output_path,"superresoluted_object"),exist_ok=True)
os.makedirs(os.path.join(output_path,"3dview_object"),exist_ok=True)
os.makedirs(os.path.join(output_path,"bbox"),exist_ok=True)

In [11]:
def object_detection_function(model,image_path,detected,cropped,box_folder):
    results=model(image_path)[0]
    
    image=cv2.imread(image_path)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    height,width,_=image.shape

    for i,box in enumerate(results.boxes):
        x1,y1,x2,y2=map(int,box.xyxy[0])
        conf=box.conf[0].item()
        new_image=image.copy()
        cv2.rectangle(new_image,(x1,y1),(x2,y2),(0,255,0),5)
        
        new_image=Image.fromarray(new_image)
        save_path=os.path.join(detected,f'{i+1}.jpg')
        new_image.save(save_path,format='JPEG')

        cropped_image=image[y1:y2,x1:x2]
        cropped_image=Image.fromarray(cropped_image)
        save_path=os.path.join(cropped,f'{i+1}.jpg')
        cropped_image.save(save_path,format='JPEG')

        bbox_filename=os.path.join(box_folder,f'{i+1}.txt')
        with open(bbox_filename,'w') as f:
            f.write(f"{x1} {y1} {x2} {y2}\n")

In [12]:
def segmentation_function(model,image_path,segmented,box_folder):
    image=cv2.imread(image_path)
    height,width,_=image.shape
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    model.set_image(image)
    
    for box_file in os.listdir(box_folder):
        box_file=os.path.join(box_folder,box_file)
        with open(box_file,'r') as f:
            line=f.readline().strip()
            x1,y1,x2,y2=map(int,line.split())

        box=np.array([x1,y1,x2,y2])
        masks,scores,logits = model.predict(
            box=box,
            multimask_output=True
        )
        mask=masks[0]
        for i in range(1,len(masks)):
            mask+=masks[i]
        # mask_image=Image.fromarray((mask*255).astype("uint8"))

        masked_image=np.zeros_like(image)
        masked_image[mask]=image[mask]
        masked_image=masked_image[y1:y2,x1:x2]
        segmented_object=Image.fromarray(masked_image)
        
        save_path=os.path.basename(box_file).replace('.txt','.png')
        save_path=os.path.join(segmented,save_path)
        segmented_object.save(save_path)

In [13]:
def inpainting_function(model,segmented,inpainted,transformer):
    model.eval()
    for segmented_image in os.listdir(segmented):
        input_data=Image.open(os.path.join(segmented,segmented_image))
        transformed_image=transformer(input_data).unsqueeze(0)
    
        with torch.no_grad():
            output=model(transformed_image)
        output=output.squeeze(0).cpu().detach().numpy()
        output=rearrange(output,'c h w-> h w c')
        output=output.clip(0,1)
        output=(output*255).astype(np.uint8)
        output=Image.fromarray(output)
        save_path=os.path.basename(segmented_image)
        save_path=os.path.join(inpainted,save_path)
        output.save(save_path)

In [14]:
def superresolution_function(upscaler,inpainted_folder,superresolution_folder):
    for inpainted_file in os.listdir(inpainted_folder):
        image=Image.open(os.path.join(inpainted_folder,inpainted_file))
        image=np.array(image)
        output,_=upscaler.enhance(image,outscale=4)
        save_path=os.path.basename(inpainted_file)
        save_path=os.path.join(superresolution_folder,save_path)
        Image.fromarray(output).save(save_path)

In [15]:
def three_d_function(model,superresolution_folder,three_d_folder):
     for superresoluted_file in os.listdir(superresolution_folder):
        image=Image.open(os.path.join(superresolution_folder,superresoluted_file))
        result_image=model(image, num_inference_steps=75).images[0]
        save_path=os.path.basename(superresoluted_file)
        save_path=os.path.join(three_d_folder,save_path)
        result_image.save(save_path)

In [16]:
detected_folder_path=os.path.join("/kaggle/working/output/detected_object")
cropped_folder_path=os.path.join("/kaggle/working/output/cropped_object")
segmented_folder_path=os.path.join("/kaggle/working/output/segmented_object")
inpainted_folder_path=os.path.join("/kaggle/working/output/inpainted_object")
superresolution_folder_path=os.path.join("/kaggle/working/output/superresoluted_object")
three_d_folder_path=os.path.join("/kaggle/working/output/3dview_object")
bbox_folder_path=os.path.join("/kaggle/working/output/bbox")

image_path=os.path.join("/kaggle/input/dataset-image-label-visiblemask-occludedmask/images/test/1222.jpg")
object_detection_function(object_detection_model,image_path,detected_folder_path,cropped_folder_path,bbox_folder_path)
segmentation_function(segmentation_model,image_path,segmented_folder_path,bbox_folder_path)
inpainting_function(inpainting_model,segmented_folder_path,inpainted_folder_path,inpainting_transformer)
superresolution_function(superresolution_upscaler,inpainted_folder_path,superresolution_folder_path)
three_d_function(zero123plus_model,superresolution_folder_path,three_d_folder_path)


image 1/1 /kaggle/input/dataset-image-label-visiblemask-occludedmask/images/test/1222.jpg: 480x640 10 items, 15.7ms
Speed: 6.7ms preprocess, 15.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

...

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

In [17]:
import shutil

shutil.make_archive("output","zip","/kaggle/working/output")

'/kaggle/working/output.zip'